From the Cell Tab > Run All. 

Run this notebook to get workspace and manifest information for all Referenced Services.

Connect to your GIS under GIS Connection.

Default export directory is C:\temp. Verify a temp folder exists in the c drive or adjust folder location. 

# Import Modules

In [ ]:
from arcgis.gis import GIS
import pandas as pd
import json
from datetime import datetime
import os

# Defined Functions

In [ ]:
def get_feature_service_url(service_url, extensions):
    """Return FeatureServer URL if extension is enabled."""
    for ext in extensions:
        if ext['typeName'] == 'FeatureServer' and ext['enabled'] == 'true':
            base_url = service_url.replace('.MapServer', '/FeatureServer').replace(f'{ags.url}', f'{rest_url}')
            return base_url
    return ""

def parse_connection_string(conn_str):
    """
    Parses an ArcGIS Server onServerConnectionString into (server, database, user).
    Returns 'Unknown' for any missing keys.
    """
    try:
        kv_pairs = [item.split('=', 1) for item in conn_str.split(';') if '=' in item]
        conn_info = {key.strip(): value.strip() for key, value in kv_pairs}
        
        server = conn_info.get('SERVER', 'Unknown').lower()
        database = conn_info.get('DATABASE', 'Unknown')
        user = conn_info.get('USER', 'Unknown')
        
        return (server, database, user)
    except Exception:
        return ('Unknown', 'Unknown', 'Unknown')

def extract_manifest_info(manifest):
    """Safely extract publishing document path and publisher."""
    try:
        resource = manifest['resources'][0]
        return resource['onPremisePath'], resource['clientName']
    except (KeyError, IndexError):
        return None, None

# GIS Connection

In [ ]:
gis = GIS(url='portal url', username="admin_user", password="password")
gis

# Script

In [ ]:
# Connect to ArcGIS Server and define variable
ags_index = 0
ags = gis.admin.servers.list()[ags_index]

#grab the AGS rest url
rest_url = gis.admin.servers.properties['servers'][ags_index]['url'] + '/rest'

print(f"Server: {ags}\n")
print(f"Rest Services Directory: {rest_url}\n")


#Grab Service workspace and manifest information

'''Add any folders you wish to excude'''

excluded_folders = {'System', 'Utilities', 'Hosted'}


records = []
total_count = 0

# --- Loop through folders and services ---
for folder in ags.services.folders:
    if folder in excluded_folders:
        continue

    for service in ags.services.list(folder=folder):
        s_url = service.url.replace(f'{ags.url}', f'{rest_url}')
        if service.serviceName == 'SampleWorldCities':
            continue

        # Format correct /MapServer URL
        s_url = s_url.replace('.MapServer', '/MapServer')
        print(s_url)

        fs_url = get_feature_service_url(service.url, service.properties.extensions)

        if service.properties.type != 'MapServer':
            continue

        s_type = service.properties.type
        s_name = service.properties.serviceName
        s_status = service.status['realTimeState']
        provider = service.properties.provider
        manifest = service.iteminformation.manifest

        publish_doc, publisher = extract_manifest_info(manifest)
        if not publish_doc:
            continue

        conn_str = manifest['databases'][0]['onServerConnectionString']
        server, db_name, db_user = parse_connection_string(conn_str)

        records.append((
            folder, s_type, s_name, s_status,
            s_url, fs_url, server, db_name, db_user,
            provider, publish_doc, publisher
        ))
        total_count += 1

# --- Output results ---
print(f"Total services processed: {total_count}")

columns = [
    'Folder', 'Type', 'Name', 'Status', "Url", "Feature Access",
    "SQL Server", "DB Name", "DB User", "Provider",
    "Publishing Document", "Publisher"
]
df = pd.DataFrame(records, columns=columns)

# Replace provider values with readable form
df['Provider'] = df['Provider'].replace({
    'DMaps': 'Pro: Shared',
    'ArcObjects11': 'Pro: Dedicated',
    'ArcObjects': 'ArcMap: Dedicated'
})

timestamp = datetime.now().strftime('%Y%m%d_%H_%M')
export_folder = r"C:\temp"
output_path = os.path.join(export_folder, f"{portal.upper()}_{timestamp}.xlsx")

df.to_excel(output_path, index=False)
df.head(10)

# Export Location

In [ ]:
print(f"Export complete: {output_path}")